# Fast FOCuS

Implementation of the fast algorithm with use of maxima checking bounds. Suitable for both Gaussian and Poisson cases.

In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from itertools import accumulate

In [74]:
class GaussianQuadratic:
    #Quadratics for the Gaussian case
    def __init__(self, a, b, τ):
        self.a = a
        self.b = b
        self.τ = τ
    
    def __repr__(self):
        return f'Quadratic: {self.a}μ^2+{np.round(self.b, 2)}μ, τ={self.τ}'
    
    def __add__(self, other_quadratic):
        return GaussianQuadratic(self.a+other_quadratic.a, self.b+other_quadratic.b, self.τ)
    
    def evaluate(self, μ):
        return np.maximum(self.a*μ**2 + self.b*μ, 0)
    
    def ymax(self):
        if self.b<=0:
            return 0
        else:
            return -self.b**2/(4*self.a) 
    
    def xmax(self):
        if self.b<=0:
            return 0
        else:
            return -self.b/(2*self.a)

def FOCuS_Gaussian(X, threshold, μ_min=0):
    min_parameter=μ_min/2
    
    curve_difference_list = [GaussianQuadratic(a=0, b=0, τ=0)]
    sum_of_maxima_list = [0]
    final_curve = GaussianQuadratic(a=0, b=0, τ=0)
    
    for T in range(len(X)):

        CTT = GaussianQuadratic(-1, 2*X[T], T)

        #adding/pruning
        if final_curve.xmax()<CTT.xmax():
            curve_difference_list.append(final_curve)
            sum_of_maxima_list.append(sum_of_maxima_list[-1]+final_curve.ymax())
            final_curve=CTT
        else:
            final_curve = final_curve+CTT
            if curve_difference_list[-1].xmax()>=final_curve.xmax():
                final_curve=curve_difference_list.pop()+final_curve
                sum_of_maxima_list.pop()
                if final_curve.xmax()<=min_parameter:
                    final_curve = GaussianQuadratic(0, 0, T+1)

        #maxima checking
        curve_being_checked=final_curve
        for i in range(len(curve_difference_list)):
            curve_max = curve_being_checked.ymax()
            if curve_max + sum_of_maxima_list[-i]<threshold:
                break
            elif curve_max >= threshold:
                return curve_being_checked.τ, T, curve_max #anomaly found
            curve_being_checked=curve_difference_list[-i]+curve_being_checked
        
    return(0, 0, None) #no anomaly found

k=5
threshold = k**2
X = stats.norm.rvs(size=1000)
FOCuS_Gaussian(X, threshold)

(0, 0, None)

In [24]:
class PoissonLogarithmCurve:
    #Logarithmic curves for the Poisson case
    def __init__(self, a, b, τ):
        self.a = a
        self.b = b
        self.τ = τ
    
    def __repr__(self):
        return f'Curve: {self.a}log(μ)-{self.b}(μ-1), τ={self.τ}'
    
    def __add__(self, other_quadratic):
        return PoissonLogarithmCurve(self.a+other_quadratic.a, self.b+other_quadratic.b, self.τ)
    
    def ymax(self):
        if self.a<=self.b:
            return 0
        else:
            return self.a*np.log(self.a/self.b)-self.a+self.b 
    
    def ymax_overestimate(self):
        return

def FOCuS_Poisson(X, λ, threshold, μ_min=1):
    if np.ndim(λ)==0:#scalar
        λ = np.full(X.shape, λ)
        
    if μ_min==1:
        min_parameter=1
    else:
        min_parameter=(μ_min-1)/np.log(μ_min)
    
    curve_difference_list = []
    sum_of_maxima_list = [0]
    final_curve = PoissonLogarithmCurve(a=0, b=0, τ=0)
    
    for T in range(len(X)):

        #adding/pruning
        if final_curve.b * X[T] > final_curve.a * λ[T]:#add new curve
            curve_difference_list.append(final_curve)
            sum_of_maxima_list.append(sum_of_maxima_list[-1]+final_curve.ymax())
            final_curve=PoissonLogarithmCurve(X[T], λ[T], T)
            
        else:#update final curve
            final_curve = PoissonLogarithmCurve(final_curve.a+X[T], final_curve.b+λ[T], final_curve.τ)
            if curve_difference_list:
                if curve_difference_list[-1].a*final_curve.b >= curve_difference_list[-1].b*final_curve.a:#prune final curve
                    final_curve=curve_difference_list.pop()+final_curve
                    sum_of_maxima_list.pop()
            if final_curve.a <= min_parameter * final_curve.b:
                final_curve = PoissonLogarithmCurve(0, 0, T+1)

        #maxima checking
        curve_being_checked=final_curve
        anomaly = None
        for i in range(len(curve_difference_list)):
            curve_max = curve_being_checked.ymax()
            if curve_max + sum_of_maxima_list[-i]<threshold:
                break #no anomaly present in rest of list
            elif curve_max >= threshold:
                #anomaly found! go through the whole curve list and report the best one
                if anomaly is None:
                    anomaly = curve_being_checked.τ, T, curve_being_checked.a/curve_being_checked.b, curve_max
                elif curve_max > anomaly[3]: #we found a better one
                    anomaly = curve_being_checked.τ, T, curve_being_checked.a/curve_being_checked.b, curve_max
            
            curve_being_checked=curve_difference_list[-i]+curve_being_checked
        
        if anomaly is not None:
                return anomaly
        
    return(0, 0, 1, 0) #no anomaly found

In [25]:
k=100
threshold = k**2/2
λ = 10
X = stats.poisson(λ).rvs(size=1000)

In [29]:
%%timeit -n2 -r1
(τ, T, μ, S) = FOCuS_Poisson(X, λ, threshold)

7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


In [10]:
%%timeit -n2 -r1
for s in range(len(X)):
    count=0
    for e in range(s, len(X)):
        count += X[e]        
        S = signif(count, (s-e+1)*λ)

C:\Users\wardk3\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in long_scalars
  
C:\Users\wardk3\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


2.94 s ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


In [35]:
def signif(a, b):
    return a*np.log(a/b)-a+b

def signif_overestimate(a, b):
    return a**2/b-b

In [38]:
signif(8, 7)

0.06825114099618013

In [39]:
signif_overestimate(8, 7)

2.1428571428571423

In [21]:
a = 8; b = 4
c = 1.5
x = (a - b)/c
y = c*(a/b)
b = x/(y-1)
a = x+b
a, b


(4.0, 1.3333333333333333)